In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
model_save_path = '/content/drive/MyDrive/KUBIG/여름CONTEST/24S NLP Basic Team 4/model/translation_model_st_jj'

tokenizer_save_path = '/content/drive/MyDrive/KUBIG/여름CONTEST/24S NLP Basic Team 4/model/translation_tokenizer_st_jj'

import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

from transformers import BartForConditionalGeneration, PreTrainedTokenizerFast

model_int_jj = BartForConditionalGeneration.from_pretrained(model_save_path)
tokenizer_int_jj = PreTrainedTokenizerFast.from_pretrained(tokenizer_save_path)
model_int_jj.to(device)

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(30000, 768, padding_idx=3)
    (encoder): BartEncoder(
      (embed_tokens): BartScaledWordEmbedding(30000, 768, padding_idx=3)
      (embed_positions): BartLearnedPositionalEmbedding(1028, 768)
      (layers): ModuleList(
        (0-5): 6 x BartEncoderLayer(
          (self_attn): BartSdpaAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): Laye

In [ ]:
# 입력 문장
input_sentence = "저리로 이렇게 너 나가있어라 하면 나가고"

# 토큰화
input_ids = tokenizer_int.encode(input_sentence, return_tensors = "pt").to(device)

# 모델 예측
output_ids = model_int.generate(input_ids, max_length = 50, num_beams = 5, early_stopping = True)

# 결과 해독
output_sentence = tokenizer_int.decode(output_ids[0], skip_special_tokens=True)
print(output_sentence)

저리로 영 너 나가서라 하면 나가고


In [ ]:
# 입력 문장
input_sentence = "전화 오는 거 같은데"

# 토큰화
input_ids = tokenizer.encode(input_sentence, return_tensors = "pt").to(device)

# 모델 예측
output_ids = model.generate(input_ids, max_length = 50, num_beams = 5, early_stopping = True)

# 결과 해독
output_sentence = tokenizer.decode(output_ids[0], skip_special_tokens=True)
print(output_sentence)

전화 오는 거 닮은디


### **KoGPT2 (All)**

In [ ]:
model_save_path = '/content/drive/MyDrive/KUBIG/여름CONTEST/24S NLP Basic Team 4/code/채원/KoGPT2(with AllData)_1st/model'

tokenizer_save_path = '/content/drive/MyDrive/KUBIG/여름CONTEST/24S NLP Basic Team 4/code/채원/KoGPT2(with AllData)_1st/tokenizer'

import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

from transformers import GPT2LMHeadModel, PreTrainedTokenizerFast

model_chatbot = GPT2LMHeadModel.from_pretrained(model_save_path)
tokenizer_chatbot = PreTrainedTokenizerFast.from_pretrained(tokenizer_save_path)
model_chatbot.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(51200, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=51200, bias=False)
)

In [ ]:
import torch
from torch.utils.data import DataLoader
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel

Q_TKN = "<usr>"
A_TKN = "<sys>"
BOS = '</s>'
EOS = '</s>'
MASK = '<unused0>'
SENT = '<unused1>'
PAD = '<pad>'

In [ ]:
with torch.no_grad():
    while 1:
        q = input("user > ").strip()
        if q == "quit":
            break
        a = ""
        while 1:
            input_ids = torch.LongTensor(tokenizer_chatbot.encode(Q_TKN + q + SENT + A_TKN + a)).unsqueeze(dim=0)
            model_chatbot = model_chatbot.to('cpu')
            pred = model_chatbot(input_ids)
            pred = pred.logits
            gen = tokenizer_chatbot.convert_ids_to_tokens(torch.argmax(pred, dim=-1).squeeze().numpy().tolist())[-1]
            if gen == EOS:
                break
            a += gen.replace("▁", " ")
        input_ids = tokenizer_int_jj.encode(a, return_tensors = "pt").to(device)
        # 모델 예측
        output_ids = model_int_jj.generate(input_ids, max_length = 50, num_beams = 5, early_stopping = True)
        # 결과 해독
        output_sentence = tokenizer_int_jj.decode(output_ids[0], skip_special_tokens=True)
        print("Chatbot > {}".format(output_sentence.strip()))

user > 요즘 날씨 어때?
Chatbot > 요즘 선선하니 딱 좋아너무 더워서 싫더라
user > 둘이 사귀면 잘 어울리겠다 으휴
Chatbot > 아주 한 쌍의 바퀴벌레가 탄생할뻔 했지 알바하면서 잘생긴 손님은 본 기억이 거의 없어너는 인
user > 난 커널스 카라멜이 젤 맛있어 아삭바삭해
Chatbot > 난 지금 배고파서 뭐든 입에 넣으면 맛있을 거 닮아
user > 그래서 넌 언제 해 
Chatbot > 나 무사 일년 전에 하지 않안
user > 밥 안 먹었더니 배고파
Chatbot > 야 너 너무 착하다난 절대 못 먹어
user > 저녁 뭐 먹을까?
Chatbot > 이제 커피는 안마셔서 추천해줘
user > 밥 뭐 먹을까?
Chatbot > 이제 막걸리에 환장하지
user > 오늘 고기 먹으러 가자 했다가 실패했어
Chatbot > 그럼 너가 맛있는 걸 사오는 건 어때
user >  그거 지금도 가능해 돈만 있으면 
Chatbot > 그래 너 돈 많다고 이것저것 다 배달 시키구 그러는 거 아니지


KeyboardInterrupt: Interrupted by user

In [ ]:
# 입력 문장
input_sentence = " 무슨 그런 게 다 있어? 신기하네 전화해봐"

# 토큰화
input_ids = tokenizer_int.encode(input_sentence, return_tensors = "pt").to(device)

# 모델 예측
output_ids = model_int.generate(input_ids, max_length = 50, num_beams = 5, early_stopping = True)

# 결과 해독
output_sentence = tokenizer_int.decode(output_ids[0], skip_special_tokens=True)
print(output_sentence)

무슨 그런 게 다 이서? 신기하네 전화해봐


In [ ]:
model_save_path = '/content/drive/MyDrive/KUBIG/여름CONTEST/24S NLP Basic Team 4/model/translation_model_st_cc'

tokenizer_save_path = '/content/drive/MyDrive/KUBIG/여름CONTEST/24S NLP Basic Team 4/model/translation_tokenizer_st_cc'

import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

from transformers import BartForConditionalGeneration, PreTrainedTokenizerFast

model_int_cc = BartForConditionalGeneration.from_pretrained(model_save_path)
tokenizer_int_cc = PreTrainedTokenizerFast.from_pretrained(tokenizer_save_path)
model_int_cc.to(device)

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(30000, 768, padding_idx=3)
    (encoder): BartEncoder(
      (embed_tokens): BartScaledWordEmbedding(30000, 768, padding_idx=3)
      (embed_positions): BartLearnedPositionalEmbedding(1028, 768)
      (layers): ModuleList(
        (0-5): 6 x BartEncoderLayer(
          (self_attn): BartSdpaAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): Laye

In [ ]:
# 입력 문장
input_sentence = "너무 좀 더럽고 그래 가지고"

# 토큰화
input_ids = tokenizer_int_cc.encode(input_sentence, return_tensors = "pt").to(device)

# 모델 예측
output_ids = model_int_cc.generate(input_ids, max_length = 50, num_beams = 5, early_stopping = True)

# 결과 해독
output_sentence = tokenizer_int_cc.decode(output_ids[0], skip_special_tokens=True)
print(output_sentence)

너무 좀 드럽고 그래 가주고


In [ ]:
with torch.no_grad():
    while 1:
        q = input("user > ").strip()
        if q == "quit":
            break
        a = ""
        while 1:
            input_ids = torch.LongTensor(tokenizer_chatbot.encode(Q_TKN + q + SENT + A_TKN + a)).unsqueeze(dim=0)
            model_chatbot = model_chatbot.to('cpu')
            pred = model_chatbot(input_ids)
            pred = pred.logits
            gen = tokenizer_chatbot.convert_ids_to_tokens(torch.argmax(pred, dim=-1).squeeze().numpy().tolist())[-1]
            if gen == EOS:
                break
            a += gen.replace("▁", " ")
        input_ids = tokenizer_int_cc.encode(a, return_tensors = "pt").to(device)
        # 모델 예측
        output_ids = model_int_cc.generate(input_ids, max_length = 50, num_beams = 5, early_stopping = True)
        # 결과 해독
        output_sentence = tokenizer_int_cc.decode(output_ids[0], skip_special_tokens=True)
        print("Chatbot > {}".format(output_sentence.strip()))

user >  아주머니 한 분 계시더라구
Chatbot > 뭐만 하면 다 돈이라고 그래 가주고 너무 싫더라
user > 그니까 그것도 던킨이 그럴 줄 몰랐어
Chatbot > 진짜    던킨이 그럴 줄 알았어
user > 오잉 ... 에콰도르 야들은 또 와 이런데
Chatbot > 왜 무슨 일이 있는 겨.


KeyboardInterrupt: Interrupted by user

### **KoGPT2 (Chatbot Data)**

In [ ]:
model_save_path = '/content/drive/MyDrive/KUBIG/여름CONTEST/24S NLP Basic Team 4/code/채원/KoGPT2 (with ChatbotData)/model_chatbotdata'

tokenizer_save_path = '/content/drive/MyDrive/KUBIG/여름CONTEST/24S NLP Basic Team 4/code/채원/KoGPT2 (with ChatbotData)/tokenizer_chatbotdata'

import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

from transformers import GPT2LMHeadModel, PreTrainedTokenizerFast

model_chatbot2 = GPT2LMHeadModel.from_pretrained(model_save_path)
tokenizer_chatbot2 = PreTrainedTokenizerFast.from_pretrained(tokenizer_save_path)
model_chatbot2.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(51200, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=51200, bias=False)
)

#### 충청도

In [ ]:
with torch.no_grad():
    while 1:
        q = input("user > ").strip()
        if q == "quit":
            break
        a = ""
        while 1:
            input_ids = torch.LongTensor(tokenizer_chatbot2.encode(Q_TKN + q + SENT + A_TKN + a)).unsqueeze(dim=0)
            model_chatbot2 = model_chatbot2.to('cpu')
            pred = model_chatbot2(input_ids)
            pred = pred.logits
            gen = tokenizer_chatbot2.convert_ids_to_tokens(torch.argmax(pred, dim=-1).squeeze().numpy().tolist())[-1]
            if gen == EOS:
                break
            a += gen.replace("▁", " ")
        input_ids = tokenizer_int_cc.encode(a, return_tensors = "pt").to(device)
        # 모델 예측
        output_ids = model_int_cc.generate(input_ids, max_length = 50, num_beams = 5, early_stopping = True)
        # 결과 해독
        output_sentence = tokenizer_int_cc.decode(output_ids[0], skip_special_tokens=True)
        print("Chatbot > {}".format(output_sentence.strip()))

user > 밥 먹었어?
Chatbot > 밥심으로 사는 거죠
user > 뭐 먹었는데?
Chatbot > 기분나쁘겠어요
user > 기분 안나쁜데
Chatbot > 정색 한번 해주세요


KeyboardInterrupt: Interrupted by user

#### 제주도

In [ ]:
with torch.no_grad():
    while 1:
        q = input("user > ").strip()
        if q == "quit":
            break
        a = ""
        while 1:
            input_ids = torch.LongTensor(tokenizer_chatbot2.encode(Q_TKN + q + SENT + A_TKN + a)).unsqueeze(dim=0)
            model_chatbot2 = model_chatbot2.to('cpu')
            pred = model_chatbot2(input_ids)
            pred = pred.logits
            gen = tokenizer_chatbot2.convert_ids_to_tokens(torch.argmax(pred, dim=-1).squeeze().numpy().tolist())[-1]
            if gen == EOS:
                break
            a += gen.replace("▁", " ")
        input_ids = tokenizer_int_jj.encode(a, return_tensors = "pt").to(device)
        # 모델 예측
        output_ids = model_int_jj.generate(input_ids, max_length = 50, num_beams = 5, early_stopping = True)
        # 결과 해독
        output_sentence = tokenizer_int_jj.decode(output_ids[0], skip_special_tokens=True)
        print("Chatbot > {}".format(output_sentence.strip()))

user > 왜 싸웠는지도 기억 안나
Chatbot > 그 이유를 찾는 과정이 되겠네요
user > 왜 싸웠는지도 기억 안나
Chatbot > 그 이유를 찾는 과정이 되겠네요
user > 갑자기 급우울
Chatbot > 낮잠을 잠깐 자도 괜찮아요
user > 연락 없더니 갑자기 왜 만나자고 하지?
Chatbot > 만나지 마세요


KeyboardInterrupt: Interrupted by user

### **KoGPT2 (final)**

In [ ]:
model_save_path = '/content/drive/MyDrive/KUBIG/여름CONTEST/24S NLP Basic Team 4/model_final'

tokenizer_save_path = '/content/drive/MyDrive/KUBIG/여름CONTEST/24S NLP Basic Team 4/tokenizer_final'

import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

from transformers import GPT2LMHeadModel, PreTrainedTokenizerFast

model_chatbot_3 = GPT2LMHeadModel.from_pretrained(model_save_path)
tokenizer_chatbot_3 = PreTrainedTokenizerFast.from_pretrained(tokenizer_save_path)
model_chatbot_3.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(51200, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=51200, bias=False)
)

In [ ]:
import torch
from torch.utils.data import DataLoader
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel

Q_TKN = "<usr>"
A_TKN = "<sys>"
BOS = '</s>'
EOS = '</s>'
MASK = '<unused0>'
SENT = '<unused1>'
PAD = '<pad>'

In [ ]:
with torch.no_grad():
    while 1:
        q = input("user > ").strip()
        if q == "quit":
            break
        a = ""
        while 1:
            input_ids = torch.LongTensor(tokenizer_chatbot.encode(Q_TKN + q + SENT + A_TKN + a)).unsqueeze(dim=0)
            model_chatbot_3 = model_chatbot_3.to('cpu')
            pred = model_chatbot_3(input_ids)
            pred = pred.logits
            gen = tokenizer_chatbot_3.convert_ids_to_tokens(torch.argmax(pred, dim=-1).squeeze().numpy().tolist())[-1]
            if gen == EOS:
                break
            a += gen.replace("▁", " ")
        input_ids = tokenizer_int_jj.encode(a, return_tensors = "pt").to(device)
        # 모델 예측
        output_ids = model_int_jj.generate(input_ids, max_length = 50, num_beams = 5, early_stopping = True)
        # 결과 해독
        output_sentence = tokenizer_int_jj.decode(output_ids[0], skip_special_tokens=True)
        print("Chatbot > {}".format(output_sentence.strip()))

user > 밥 먹었니?
Chatbot > 응응 오늘 밥먹언
user > 뭐 먹었어?
Chatbot > 나 햄버거 먹언
user > 나는 뭐 먹었게?
Chatbot > 나 햄버거
user > 난 커널스 카라멜이 젤 맛있어 아삭바삭해
Chatbot > 난 지금 배고파서 그냥 그랬어
user > 지금 비 오는 것 같아?
Chatbot > 지금 비 오는 거 아니야
user > 그래서 넌 언제 해?
Chatbot > 난 요즘 하는 거 없이 바로바로 하고 인
user > 오늘 세상에 무슨 일 있니?
Chatbot > 무슨 일 이서
user > 헐 그거 봤어?
Chatbot > 그거 시즌투도 있어
user > 디즈니 좋아하시나요?
Chatbot > 디즈니 좋아하는 거 같애
user > 따뜻한 차 좋아해?
Chatbot > 나는 차가운 차가 더 좋아
user > 따뜻한 건?
Chatbot > 차가운 건 아닌데
user > quit


In [ ]:
with torch.no_grad():
    while 1:
        q = input("user > ").strip()
        if q == "quit":
            break
        a = ""
        while 1:
            input_ids = torch.LongTensor(tokenizer_chatbot.encode(Q_TKN + q + SENT + A_TKN + a)).unsqueeze(dim=0)
            model_chatbot_3 = model_chatbot_3.to('cpu')
            pred = model_chatbot_3(input_ids)
            pred = pred.logits
            gen = tokenizer_chatbot_3.convert_ids_to_tokens(torch.argmax(pred, dim=-1).squeeze().numpy().tolist())[-1]
            if gen == EOS:
                break
            a += gen.replace("▁", " ")
        input_ids = tokenizer_int_cc.encode(a, return_tensors = "pt").to(device)
        # 모델 예측
        output_ids = model_int_cc.generate(input_ids, max_length = 50, num_beams = 5, early_stopping = True)
        # 결과 해독
        output_sentence = tokenizer_int_cc.decode(output_ids[0], skip_special_tokens=True)
        print("Chatbot > {}".format(output_sentence.strip()))

user > 밥 먹었니?
Chatbot > 응응 오늘 밥먹었어
user > 나 며칠 전에 독립하고 싶어서 집 알아보고 왔음
Chatbot > 독립하면 집도 엄청 비싸지 않아
user >  진짜 독립을 하려는 건 아니고
Chatbot > 근데 왜 부동산까지 가서 알아보는 겨
user > 알바하고 많이 힘들었구나
Chatbot > 응   힘들었지만 열심히 했지
user > 나는 지금 학원 선택 때문에 고민이 많아
Chatbot > 왜  어떤 과목을 선택하는데
user > 너는 수상 스키 배워보고 싶어?
Chatbot > 아니 난 수상스키 배워보고 싶어
user > 그래 그러면 그냥 이번 여름에 너랑 수상 스키나 배워야겠다
Chatbot > 그래 같이 배워보자
user > 여수 밤바다 보면서 술 마시는 곳이래
Chatbot > 거기 진짜 좋겠다 난 여수 밤바다 보면서 술 한 잔도 못 마심
user > 내가 게임 너무 달달 연습했나 봐
Chatbot > 아니야   나는 연습을 많이 했었거든
user > 오늘 이케아 가서 소품이랑 장식들 사서 놓으려고
Chatbot > 이케아에서 어떤 거 샀어
user > 곧 여름이니까 스타일도 시원하게 바꾸려고
Chatbot > 시원하게 남자들 많이 하는 거 있잖아
user > quit
